<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/Destilacio%CC%81n1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def calcular_constantes_equilibrio(temperaturas, alpha, beta, gamma):
    # Calcular las constantes de equilibrio K_ji para cada componente en cada etapa
    K = np.zeros((len(temperaturas), len(alpha)))
    for j, T in enumerate(temperaturas):
        for i in range(len(alpha)):
            K[j, i] = alpha[i] + beta[i] * T + gamma[i] * T**2
    return K

def resolver_sistema_lineal_rectificacion(num_etapas, A0, razones_adsorcion):
    # Inicializar la matriz A y el vector b para Ax = b en la zona de rectificación
    A = np.zeros((num_etapas, num_etapas))
    b = np.ones(num_etapas)
    b[0] = A0 + 1  # Establecer el valor para la primera ecuación (condensador)

    # Llenar la matriz bidiagonal A para la zona de rectificación
    for j in range(num_etapas):
        if j == 0:
            A[j, j] = 1  # Primer elemento diagonal para el condensador
        else:
            A[j, j] = 1  # Elemento diagonal
            A[j, j-1] = -razones_adsorcion[j-1]  # Elemento subdiagonal

    # Resolver el sistema lineal
    x = np.linalg.solve(A, b)

    return x

def resolver_sistema_lineal_stripping(num_etapas, A0, razones_adsorcion):
    # Inicializar la matriz A y el vector b para Ax = b en la zona de stripping
    A = np.zeros((num_etapas, num_etapas))
    b = np.ones(num_etapas)
    b[0] = A0 + 1  # Establecer el valor para la primera ecuación (rehervidor)

    # Llenar la matriz bidiagonal A para la zona de stripping
    for j in range(num_etapas):
        if j == 0:
            A[j, j] = 1  # Primer elemento diagonal para el rehervidor
        else:
            A[j, j] = 1  # Elemento diagonal
            A[j, j-1] = -razones_adsorcion[j-1]  # Elemento subdiagonal

    # Resolver el sistema lineal
    x = np.linalg.solve(A, b)

    return x

# Datos dados
F = 1000  # mol/h (Caudal de alimentación)
z = [0.06, 0.17, 0.22, 0.20, 0.35]  # Fracciones molares de alimentación para los componentes 1 a 5
relacion_reflujo = 2.5  # L1 / D
temperaturas_F = [140, 150, 160, 170, 180, 190, 200, 210, 220, 230]  # Temperaturas en °F
temperaturas_K = [(T - 32) * 5/9 + 273.15 for T in temperaturas_F]  # Convertir a Kelvin para consistencia

# Parámetros de las constantes de equilibrio para cada componente (de la Tabla P2.5)
alpha = [0.70, 2.21, 1.50, 0.86, 0.71]
beta = [0.30e-2, 1.95e-2, -1.60e-2, -0.97e-2, -0.87e-2]
gamma = [0.65e-4, 0.90e-4, 0.80e-4, 0.46e-4, 0.42e-4]

# Paso 1: Calcular las constantes de equilibrio
K = calcular_constantes_equilibrio(temperaturas_F, alpha, beta, gamma)

#Sacar L y V
D=500
L_D=2.5
L=D*L_D
V=D+L

# Paso 2: Calcular los caudales de rectificación y stripping

# Definir los parámetros de las zonas de rectificación y stripping
num_etapas_rectificacion = 5  # Número de etapas en la zona de rectificación
num_etapas_stripping = 5  # Número de etapas en la zona de stripping
A0_rectificacion = L/D  # Valor inicial para la razón de adsorción en el condensador
A0_stripping = L/D  # Valor inicial para la razón de adsorción en el rehervidor
razones_adsorcion_rectificacion = [0.5, 0.6, 0.7, 0.8]  # Ejemplo de razones de adsorción para la rectificación
razones_adsorcion_stripping = [0.7, 0.8, 0.9, 1.0]  # Ejemplo de razones de adsorción para la stripping

# Resolver el sistema lineal para la zona de rectificación
caudales_rectificacion = resolver_sistema_lineal_rectificacion(num_etapas_rectificacion, A0_rectificacion, razones_adsorcion_rectificacion)

# Resolver el sistema lineal para la zona de stripping
caudales_stripping = resolver_sistema_lineal_stripping(num_etapas_stripping, A0_stripping, razones_adsorcion_stripping)

def valores_a(K):
# Hacemos una lista para almacenar los valores de a
    a_values = np.zeros_like(K)

# Calculamos los valores de a para cada Kij
    for i in range(K.shape[0]):  # Itera sobre las filas
       for j in range(0,K.shape[1]):  # Itera sobre las columnas
              a_values[i, j] = (L / (V * K[i, j]))
    extra_row = np.full((1, K.shape[1]), L_D)

    a_values = np.vstack([extra_row, a_values])
    return a_values
# Mostramos los resultados
P=valores_a(K)
print("\nValores de a:")
print(P)


# Mostrar los resultados numéricos
print("\nConstantes de equilibrio K_ji por etapa y componente:")
print(K)

print("\nCaudales de rectificación (v_ji / d_i) para cada etapa:")
for i, caudal in enumerate(caudales_rectificacion, start=1):
    print(f"Etapa {i}: {caudal:.4f}")

print("\nCaudales de stripping (v_ji / d_i) para cada etapa:")
for i, caudal in enumerate(caudales_stripping, start=1):
    print(f"Etapa {i}: {caudal:.4f}")

# Graficar los resultados de los caudales de stripping y rectificación
etapas_rectificacion = np.arange(1, num_etapas_rectificacion + 1)
etapas_stripping = np.arange(1, num_etapas_stripping + 1)

# Función que convierte cada columna de la matriz en un vector separado
def dematrizavector(M):# M es la matriz que se crea
    # Obtener las dimensiones de la matriz
    n, m = M.shape

    # Se crea una lista que tenga vectores que tengan una longitud m
    vectores = [M[:, j]
                for j in range(m)]

    return vectores

# Transformar la matriz en vectores
vectores = dematrizavector(P)

# Mostrar los vectores resultantes
for i, vec in enumerate(vectores):
    print(f"Vector {i+1}: {vec}")


def generar_sistema_ecuaciones_alternativo(v):
    # Convertir el vector de NumPy a lista, si es necesario
    v = v.tolist() if isinstance(v, np.ndarray) else v

    # Longitud del vector
    n = len(v)

    # Definir las incógnitas simbólicas x1, x2, ..., xn
    x = sp.symbols(f'x1:{n+1}')

    # Inicializar lista vacía de ecuaciones
    ecuaciones = []

    # Primera ecuación: x1 = vector[0] + 1
    primera_ecuacion = sp.Eq(x[0], v[0] + 1)
    ecuaciones.append(primera_ecuacion)

    # Generar las siguientes ecuaciones usando un bucle for
    for i in range(1, n):
        ecuaciones.append(sp.Eq(x[i], v[i] * x[i-1] + 1))

    return ecuaciones, x

"Lo de chat"


# Convertir las ecuaciones simbólicas en una matriz de coeficientes y un vector de constantes
def convertir_ecuaciones_a_matriz_alternativo(ecuacion, variables):
    n = len(variables)
    A = np.zeros((n, n))
    b = np.zeros(n)

    for i, eq in enumerate(ecuacion):
        lhs, rhs = eq.lhs, eq.rhs
        b[i] = float(rhs.subs({var:0 for var in variables})) # Set all variables to 0 for evaluation

        # Extraer los coeficientes de las incógnitas
        for var in variables:
            if var in lhs.free_symbols:
                A[i, variables.index(var)] = float(lhs.coeff(var))

    return A, b

# Combinar la matriz de coeficientes y el vector en una matriz extendida
def crear_matriz_extendida(A, b):
    C=np.hstack((A, b.reshape(-1, 1)))
    return C

# Implementar el algoritmo de eliminación gaussiana
def eliminacion_gaussiana(matriz):
    n = len(matriz)
    for i in range(n):
        # Intercambiar filas para evitar divisiones por cero
        max_fila = max(range(i, n), key=lambda k: abs(matriz[k][i]))
        matriz[[i, max_fila]] = matriz[[max_fila, i]]

        # Normalizar la fila pivote
        matriz[i] = matriz[i] / matriz[i][i]

        # Eliminar los elementos debajo del pivote
        for j in range(i + 1, n):
            matriz[j] = matriz[j] - matriz[i] * matriz[j][i]

    # Sustitución hacia atrás
    for i in range(n-1, -1, -1):
        for j in range(i-1, -1, -1):
            matriz[j] = matriz[j] - matriz[i] * matriz[j][i]

    return matriz

# Definir los vectores originales
vectores = [
    np.array([2.5, 0.45964332, 0.41710115, 0.37913254, 0.34531579,
              0.31521876, 0.28842549, 0.26455026, 0.2432439, 0.22419514, 0.20712939]),
    np.array([2.5, 0.57418466, 0.54525627, 0.51240008, 0.47746371,
              0.44200849, 0.40723245, 0.37397158, 0.34274746, 0.31383379, 0.2873233]),
    np.array([2.5, 0.13456777, 0.12531328, 0.11694265, 0.10935176,
              0.10245062, 0.09616124, 0.09041591, 0.08515566, 0.08032903, 0.07589096]),
    np.array([2.5, 0.22896708, 0.21321962, 0.19898755, 0.18608944,
              0.17436913, 0.16369184, 0.15394089, 0.14501497, 0.13682586, 0.12929652]),
    np.array([2.5, 0.25962697, 0.24131274, 0.2248161, 0.20991117,
              0.196405, 0.18413222, 0.17295054, 0.16273711, 0.15338553, 0.1448033]),
]

# Iterar a través de cada vector y resolver el sistema
for idx, vector in enumerate(vectores, start=1):
    # Generar el sistema de ecuaciones
    sistema, variables = generar_sistema_ecuaciones_alternativo(vector)

    # Convertir el sistema en matriz y vector
    A, b = convertir_ecuaciones_a_matriz_alternativo(sistema, variables)

    # Crear la matriz extendida [A | b]
    matriz_extendida = crear_matriz_extendida(A, b)

    # Mostrar la matriz A, el vector b y la matriz extendida
    print(f"\nMatriz de coeficientes A para el Vector {idx}:")
    print(A)
    print(f"\nVector de constantes b para el Vector {idx}:")
    print(b)
    print(f"\nMatriz extendida [A | b] para el Vector {idx}:")
    print(matriz_extendida)

    # Aplicar eliminación gaussiana
    resultado = eliminacion_gaussiana(matriz_extendida.copy())
    print(f"\nMatriz tras la eliminación gaussiana para el Vector {idx}:")
    print(resultado)


# Función para realizar la sustitución hacia atrás
def sustitucion_hacia_atras(matriz):
    filas, columnas = matriz.shape
    x = np.zeros(filas)

    # Aplicar sustitución hacia atrás
    for i in range(filas - 1, -1, -1):
        x[i] = matriz[i, -1] / matriz[i, i]
        for j in range(i - 1, -1, -1):
            matriz[j, -1] -= matriz[j, i] * x[i]

    return x

# Función general para resolver y almacenar valores de incógnitas
def resolver_valores_x(vectores, indices, descripcion=""):
    resultados_x = []

    for idx, vector in enumerate(vectores, start=1):
        # Generar el sistema de ecuaciones
        sistema, variables = generar_sistema_ecuaciones(vector, indices)

        # Convertir el sistema en una matriz de coeficientes y vector de constantes
        A, b = ecuaciones_a_matriz(sistema, variables)
        matriz_extendida = juntar_matriz_y_vector(A, b)

        # Aplicar eliminación gaussiana
        resultado = eliminacion_gaussiana(matriz_extendida)

        # Obtener los valores de x_ij mediante sustitución hacia atrás
        resultado_np = np.array(resultado)
        valores_x = sustitucion_hacia_atras(resultado_np)
        resultados_x.append(valores_x)

        # Mostrar los valores de las incógnitas x_ij
        print(f"\nValores de las incógnitas x_ij para el Vector {idx} {descripcion}:")
        for j, valor in enumerate(valores_x, start=indices[0] + 1):
            print(f"x{j} = {valor}")

    return resultados_x

# Resolver para los índices de 1 a 5
indices_1_5 = list(range(0, 5))  # Indices de 1 a 5
valores_x_1_5 = resolver_valores_x(vectores, indices_1_5, descripcion="(índices 1 a 5)")

# Resolver para los índices de 7 a 11
indices_7_11 = list(range(6, 11))  # Indices de 7 a 11
valores_x_7_11 = resolver_valores_x(vectores, indices_7_11, descripcion="(índices 7 a 11)")



Valores de a:
[[2.5        2.5        2.5        2.5        2.5       ]
 [0.29836496 0.1065462  0.86266391 1.76978621 2.26613488]
 [0.2734108  0.09976057 0.79365079 1.62337662 2.04081633]
 [0.25115531 0.09356638 0.72296125 1.47093434 1.81659642]
 [0.23127269 0.08790127 0.6541078  1.32177223 1.60585817]
 [0.21347451 0.08271025 0.58934465 1.18180959 1.41498755]
 [0.19750746 0.07794475 0.52988554 1.05414066 1.24613698]
 [0.18315018 0.07356187 0.47619048 0.93984962 1.0989011 ]
 [0.17020987 0.06952362 0.42822885 0.8387573  0.97155293]
 [0.1585188  0.0657964  0.38568343 0.749985   0.86183122]
 [0.14793118 0.06235036 0.34809245 0.67233219 0.76738904]]

Constantes de equilibrio K_ji por etapa y componente:
[[ 2.394   6.704   0.828   0.4036  0.3152]
 [ 2.6125  7.16    0.9     0.44    0.35  ]
 [ 2.844   7.634   0.988   0.4856  0.3932]
 [ 3.0885  8.126   1.092   0.5404  0.4448]
 [ 3.346   8.636   1.212   0.6044  0.5048]
 [ 3.6165  9.164   1.348   0.6776  0.5732]
 [ 3.9     9.71    1.5     0.76  

NameError: name 'generar_sistema_ecuaciones' is not defined